Notebook for K Neighbour Regression model

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

# create a default KNR pipelinei with standardised scaling
default_knr = Pipeline([
    ('scaler', StandardScaler()),
    ('knr', KNeighborsRegressor())
])

10-fold cross-validation used on full dataset to evaluate mean train and test metrics

In [58]:
from sklearn.model_selection import KFold, cross_validate
import numpy as np
%store -r X y

scoring = {
    "neg_mse": "neg_mean_squared_error",
    "neg_mae": "neg_mean_absolute_error"
}

# 10 fold cross validation setup 
ten_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# cross validate using default knr
default_cv = cross_validate(
    default_knr, 
    X, y, 
    cv=ten_fold_cv,
    scoring=scoring,
    return_train_score=True
)

# check available keys
print(default_cv.keys())


# convert corrospinding negative errors to positive
train_mse = -default_cv["train_neg_mse"]
train_rmse = np.sqrt(train_mse).mean()              # get mean rmse over 10 folds

test_mse  = -default_cv["test_neg_mse"]
test_rmse = np.sqrt(test_mse).mean()

train_mae = -default_cv["train_neg_mae"]
test_mae  = -default_cv["test_neg_mae"]

# print default knr results
print("Default KNR mean train RMSE:", train_rmse)
print("Default KNR mean test  RMSE:", test_rmse)

print("Default KNR mean train MAE:", train_mae.mean())
print("Default KNR mean test  MAE:", test_mae.mean())
                                    

dict_keys(['fit_time', 'score_time', 'test_neg_mse', 'train_neg_mse', 'test_neg_mae', 'train_neg_mae'])
Default KNR mean train RMSE: 34.3076604870604
Default KNR mean test  RMSE: 42.65590640074778
Default KNR mean train MAE: 26.19680793929502
Default KNR mean test  MAE: 33.01488942465785


Find optimal hyperparameters for 'n_neighbors' and 'weights' using GridSearchCV

In [59]:
from sklearn.model_selection import GridSearchCV

# hp tuning using GridSearchCV
grid_knr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('tuned_knr', KNeighborsRegressor())
])

# enter range of hyperparameters to test (prefix hp's with model name in pipeline)
param_grid = {
    'tuned_knr__n_neighbors': [3, 5, 10, 15, 20, 30, 50, 60, 70], 
    'tuned_knr__weights': ['uniform', 'distance']
}

# split the dataset and perform grid search with 10 fold cv
gridsearch_knr = GridSearchCV(grid_knr_pipeline, param_grid, cv= 10, scoring='neg_mean_squared_error')
gridsearch_knr.fit(X, y)

best_params = gridsearch_knr.best_params_
print(f"Best hyperparameters: {best_params}")


Best hyperparameters: {'tuned_knr__n_neighbors': 5, 'tuned_knr__weights': 'distance'}


Apply improved hyperparameters to new model

In [ ]:

improved_knr = Pipeline([
    ('scaler', StandardScaler()),
    ('improved_knr', KNeighborsRegressor(n_neighbors=5, weights='distance'))
])


# 10 fold cross validation setup 
ten_fold_cv_improved = KFold(n_splits=10, shuffle=True, random_state=42)

# cross validate using optimal knr
improved_cv = cross_validate(
    improved_knr, 
    X, y, 
    cv=ten_fold_cv_improved,
    scoring=scoring,
    return_train_score=True
)


# convert corrospinding negative errors to positive
improved_train_mse = -improved_cv["train_neg_mse"]
improved_train_rmse = np.sqrt(improved_train_mse).mean()              # get mean rmse over 10 folds

improved_test_mse  = -improved_cv["test_neg_mse"]
improved_test_rmse = np.sqrt(improved_test_mse).mean()

improved_train_mae = -improved_cv["train_neg_mae"]
improved_test_mae  = -improved_cv["test_neg_mae"]

# print default knr results
print("Improved KNR mean train RMSE:", improved_train_rmse)
print("Improved KNR mean test  RMSE:", improved_test_rmse)

print("Improved KNR mean train MAE:", improved_train_mae.mean())
print("Improved KNR mean test  MAE:", improved_test_mae.mean())
                                    

dict_keys(['fit_time', 'score_time', 'test_neg_mse', 'train_neg_mse', 'test_neg_mae', 'train_neg_mae'])
Improved KNR mean train RMSE: 0.0
Improved KNR mean test  RMSE: 39.660960336794496
Improved KNR mean train MAE: 0.0
Improved KNR mean test  MAE: 30.185019775135448
